# QTN (json)

In [1]:
qtn_file_path = '/content/taskVehicleStore_c5c6161f-0ab2-410a-9f6d-6a14c7a3c3c2.json'
vehicle_config = "notAllVehicle" #use all vehicle / not use all vehicle

In [ ]:
import pandas as pd
import json

# Load the JSON file or string
with open(qtn_file_path) as f:
    data = json.load(f)

# Flatten all stores with their tour ID
store_rows = []
for tour_id, tour_info in data.items():
    for store in tour_info.get('stores', []):
        store_copy = store.copy()
        store_copy["tour_id"] = tour_id
        store_rows.append(store_copy)

# Create a DataFrame
df = pd.DataFrame(store_rows)

# Save to Excel
df.to_excel("stores_output.xlsx", index=False)

In [ ]:
#from individual stores (primarily to check if transaction/stop duration is the same; it's actually a set time based on the algo, so it wont fluctuate much)

import pandas as pd

# Load the data
df = pd.read_excel("stores_output.xlsx")

# Convert time columns
df['eta'] = pd.to_datetime(df['eta'])
df['etd'] = pd.to_datetime(df['etd'])

# Compute per-task duration
df['TASK_DURATION_MIN'] = (df['etd'] - df['eta']).dt.total_seconds() / 60

# --- 1. Total working time per tour: span between first ETA and last ETD ---
tour_times = df.groupby('tour_id').agg(
    TOUR_START=('eta', 'min'),
    TOUR_END=('etd', 'max'),
    TOTAL_STOP_DURATION_MIN=('TASK_DURATION_MIN', 'sum')
).reset_index()

# Compute total working span
tour_times['TOUR_WORKING_SPAN_MIN'] = (tour_times['TOUR_END'] - tour_times['TOUR_START']).dt.total_seconds() / 60

# --- 2. Travel time = span - stop durations
tour_times['TRAVEL_TIME_MIN'] = tour_times['TOUR_WORKING_SPAN_MIN'] - tour_times['TOTAL_STOP_DURATION_MIN']

# Display
print(tour_times[['tour_id', 'TOUR_WORKING_SPAN_MIN', 'TOTAL_STOP_DURATION_MIN', 'TRAVEL_TIME_MIN']])

total_working_span = tour_times['TOUR_WORKING_SPAN_MIN'].sum()
total_stop_duration = tour_times['TOTAL_STOP_DURATION_MIN'].sum()
total_travel_time = tour_times['TRAVEL_TIME_MIN'].sum()

# Sum distance per tour
tour_distances = df.groupby('tour_id')['distance'].sum().reset_index()
tour_distances.rename(columns={'distance': 'TOTAL_DISTANCE_KM'}, inplace=True)

# Merge with previous tour_times DataFrame
tour_summary = pd.merge(tour_times, tour_distances, on='tour_id', how='left')

# Print per-tour summary
print(tour_summary[['tour_id', 'TOUR_WORKING_SPAN_MIN', 'TOTAL_STOP_DURATION_MIN', 'TRAVEL_TIME_MIN', 'TOTAL_DISTANCE_KM']])

# Grand totals
total_distance = tour_summary['TOTAL_DISTANCE_KM'].sum()

print("\n--- GRAND TOTALS ---")
print(f"Total Working Span: {tour_summary['TOUR_WORKING_SPAN_MIN'].sum():.2f} minutes")
print(f"Total Stop Duration: {tour_summary['TOTAL_STOP_DURATION_MIN'].sum():.2f} minutes")
print(f"Total Travel Time: {tour_summary['TRAVEL_TIME_MIN'].sum():.2f} minutes")
print(f"Total Distance: {total_distance:.2f} km")


                                tour_id  TOUR_WORKING_SPAN_MIN  \
0  05af0c78-2c6a-47db-8a7f-d91cb71fe6b3                  424.0   
1  1e12c471-dae5-4a8a-9f56-221d7d051176                  231.0   
2  327d2f3d-4dbb-4c35-b82f-ff39bfb50b90                  390.0   
3  5f02778c-b384-4360-a819-c534638313dc                  206.0   

   TOTAL_STOP_DURATION_MIN  TRAVEL_TIME_MIN  
0                    244.0            180.0  
1                    135.0             96.0  
2                    215.0            175.0  
3                    140.0             66.0  
                                tour_id  TOUR_WORKING_SPAN_MIN  \
0  05af0c78-2c6a-47db-8a7f-d91cb71fe6b3                  424.0   
1  1e12c471-dae5-4a8a-9f56-221d7d051176                  231.0   
2  327d2f3d-4dbb-4c35-b82f-ff39bfb50b90                  390.0   
3  5f02778c-b384-4360-a819-c534638313dc                  206.0   

   TOTAL_STOP_DURATION_MIN  TRAVEL_TIME_MIN  TOTAL_DISTANCE_KM  
0                    244.0            180.0

In [ ]:
# with sum at the end, also added weight, item, etc

# from tour

import json
from datetime import datetime
import pandas as pd

# Load JSON data
with open(qtn_file_path) as f:
    data = json.load(f)

results = []

for key, value in data.items():
    tour = value.get("tour", {})

    tour_id = tour.get("code", key)
    tour_start = tour.get("tourStartDate")
    tour_end = tour.get("tourEndDate")
    travel_time = tour.get("totalDuration", 0)  # in minutes
    total_distance = tour.get("totalDistance", 0) / 1000  # meters to km

    total_order = tour.get("totalOrder", 0)
    total_qty = tour.get("totalQty", 0)
    total_volume = tour.get("totalVolume", 0.0)
    total_weight = tour.get("totalWeight", 0.0) * 1000 # ton to kg
    vehicle_model_code = tour.get("vehicleModelCode", "Unknown")

    # Convert to datetime
    try:
        start_dt = datetime.fromisoformat(tour_start.replace("Z", "+00:00"))
        end_dt = datetime.fromisoformat(tour_end.replace("Z", "+00:00"))
        total_working_time = (end_dt - start_dt).total_seconds() / 60  # in minutes
    except Exception:
        total_working_time = None

    transaction_time = total_working_time - travel_time if total_working_time is not None else None

    results.append({
        "tour_id": tour_id,
        "vehicle_model_code": vehicle_model_code,
        "total_working_time_min": round(total_working_time, 2) if total_working_time is not None else None,
        "transaction_time_min": round(transaction_time, 2) if transaction_time is not None else None,
        "travel_time_min": round(travel_time, 2),
        "total_distance_km": round(total_distance, 2),
        "total_order": total_order,
        "total_qty": total_qty,
        "total_volume": total_volume,
        "total_weight": total_weight
    })

# Convert to DataFrame
df = pd.DataFrame(results)

# Reorder columns
ordered_columns = [
    "tour_id",
    "vehicle_model_code",
    "total_order",
    "total_qty",
    "total_weight",
    "total_volume",
    "total_working_time_min",
    "transaction_time_min",
    "travel_time_min",
    "total_distance_km"
]
df = df[ordered_columns]

# Add Grand Total row
totals = {
    "tour_id": "TOTAL",
    "vehicle_model_code": "MIXED",
    "total_order": df["total_order"].sum(),
    "total_qty": df["total_qty"].sum(),
    "total_weight": df["total_weight"].sum(),
    "total_working_time_min": df["total_working_time_min"].sum(),
    "transaction_time_min": df["transaction_time_min"].sum(),
    "travel_time_min": df["travel_time_min"].sum(),
    "total_distance_km": df["total_distance_km"].sum(),
    "total_volume": df["total_volume"].sum()
}
df = pd.concat([df, pd.DataFrame([totals])], ignore_index=True)

# Save to Excel
df.to_excel(f"qtn_data_{vehicle_config}.xlsx", index=False)

# Print Grand Totals
print("\n--- Grand Totals ---")
print(f"Total Working Time: {totals['total_working_time_min']:.2f} minutes")
print(f"Total Transaction Time: {totals['transaction_time_min']:.2f} minutes")
print(f"Total Travel Time: {totals['travel_time_min']:.2f} minutes")
print(f"Total Distance: {totals['total_distance_km']:.2f} km")
print(f"Total Orders: {totals['total_order']}")
print(f"Total Quantity: {totals['total_qty']}")
print(f"Total Volume: {totals['total_volume']:.6f}")
print(f"Total Weight: {totals['total_weight']:.6f} kg")



--- Grand Totals ---
Total Working Time: 1636.00 minutes
Total Transaction Time: 734.00 minutes
Total Travel Time: 902.00 minutes
Total Distance: 560.09 km
Total Orders: 221
Total Quantity: 4798
Total Volume: 14.719299
Total Weight: 5476.245510 kg


# Locus (merged excel)

In [ ]:
import pandas as pd

# Load the Excel file
df = pd.read_excel('/content/locus_notuseallvehicle_cleanmergeall.xlsx')  # Adjust path as needed

# Add a column for total weight per row
df['total_weight'] = df['QUANTITY_y'] * df['WEIGHT'] / 1000

# Group by TOUR_ID to get per-tour metrics
per_tour_summary = df.groupby('TOUR_ID').agg({
    'TASK_ID': 'nunique',                       # total_order
    'QUANTITY_y': 'sum',                        # total_qty
    'VOLUME_y': 'sum',                          # total_volume
    'total_weight': 'sum',                      # total_weight
    'TOUR_TRAVEL_DISTANCE': 'first',
    'TOUR_TRAVEL_DURATION': 'first',
    'TOUR_TRANSACTION_DURATION': 'first',
    'VEHICLE_MODEL': 'first'
}).reset_index()

# Rename columns for consistency
per_tour_summary = per_tour_summary.rename(columns={
    'TOUR_ID': 'tour_id',
    'TASK_ID': 'total_order',
    'QUANTITY_y': 'total_qty',
    'VOLUME_y': 'total_volume',
    'TOUR_TRAVEL_DISTANCE': 'total_distance_km',
    'TOUR_TRAVEL_DURATION': 'travel_time_min',
    'TOUR_TRANSACTION_DURATION': 'transaction_time_min',
    'VEHICLE_MODEL': 'vehicle_model_code'
})

# Add total working time column
per_tour_summary['total_working_time_min'] = (
    per_tour_summary['travel_time_min'] + per_tour_summary['transaction_time_min']
)

# Reorder columns as specified
ordered_columns = [
    'tour_id',
    'vehicle_model_code',
    'total_order',
    'total_qty',
    'total_weight',
    'total_volume',
    'total_working_time_min',
    'transaction_time_min',
    'travel_time_min',
    'total_distance_km'
]
per_tour_summary = per_tour_summary[ordered_columns]

# Compute grand totals
totals = {
    'tour_id': 'TOTAL',
    'vehicle_model_code': 'MIXED',
    'total_order': per_tour_summary['total_order'].sum(),
    'total_qty': per_tour_summary['total_qty'].sum(),
    'total_weight': per_tour_summary['total_weight'].sum(),
    'total_working_time_min': per_tour_summary['total_working_time_min'].sum(),
    'transaction_time_min': per_tour_summary['transaction_time_min'].sum(),
    'travel_time_min': per_tour_summary['travel_time_min'].sum(),
    'total_distance_km': per_tour_summary['total_distance_km'].sum(),
    'total_volume': per_tour_summary['total_volume'].sum()
}

# Append totals row
per_tour_summary_with_total = pd.concat(
    [per_tour_summary, pd.DataFrame([totals])],
    ignore_index=True
)

# Save to Excel
per_tour_summary_with_total.to_excel(f"locus_data_{vehicle_config}.xlsx", index=False)

# Optional: print to console
print("=== Per-Tour Summary ===")
print(per_tour_summary)

print("\n=== Grand Totals ===")
print(pd.DataFrame([totals]))


=== Per-Tour Summary ===
  tour_id vehicle_model_code  total_order  total_qty  total_weight  \
0  tour-1                CDD           22        375   1245.939940   
1  tour-2                CDD           72       1555   1824.424084   
2  tour-3                CDD           54       1014   1244.244660   
3  tour-4                CDE           71       1829   1153.740840   

   total_volume  total_working_time_min  transaction_time_min  \
0       6.35850                     334                   145   
1       6.59252                     594                   224   
2       3.83503                     367                   135   
3       4.10585                     582                   221   

   travel_time_min  total_distance_km  
0              189              91.02  
1              370             184.34  
2              232             117.99  
3              361             182.37  

=== Grand Totals ===
  tour_id vehicle_model_code  total_order  total_qty  total_weight  \
0   TO

In [ ]:
quantum_df = pd.read_excel(f"/content/qtn_data_{vehicle_config}.xlsx")
locus_df = pd.read_excel(f"/content/locus_data_{vehicle_config}.xlsx")

In [ ]:
import pandas as pd

# Extract TOTAL rows
quantum_total = quantum_df[quantum_df['tour_id'] == 'TOTAL'].squeeze()
locus_total = locus_df[locus_df['tour_id'] == 'TOTAL'].squeeze()

# List of metrics to compare
metrics = [
    "total_working_time_min",
    "transaction_time_min",
    "travel_time_min",
    "total_distance_km",
    "total_order",
    "total_qty",
    "total_volume",
    "total_weight"
]

# Build comparison data
comparison_data = {
    "Metric": [],
    "Quantum": [],
    "Locus": []
}

for metric in metrics:
    quantum_value = quantum_total.get(metric, None)
    locus_value = locus_total.get(metric, None)

    # Round numeric values to 2 decimals
    if isinstance(quantum_value, (int, float)):
        quantum_value = round(quantum_value, 2)
    if isinstance(locus_value, (int, float)):
        locus_value = round(locus_value, 2)

    comparison_data["Metric"].append(metric)
    comparison_data["Quantum"].append(quantum_value)
    comparison_data["Locus"].append(locus_value)

# Create DataFrame
comparison_df = pd.DataFrame(comparison_data)

# Save to Excel
comparison_df.to_excel("/content/quantum_vs_locus_comparison.xlsx", index=False)

# Optional: print preview
print(comparison_df)


                   Metric  Quantum    Locus
0  total_working_time_min  1636.00  1877.00
1    transaction_time_min   734.00   725.00
2         travel_time_min   902.00  1152.00
3       total_distance_km   560.09   575.72
4             total_order   221.00   219.00
5               total_qty  4798.00  4773.00
6            total_volume    14.72    20.89
7            total_weight  5476.25  5468.35


In [ ]:
# Define which metrics are "lower" or "higher" is better
direction_rules = {
    "total_working_time_min": "lower",  # lower is better
    "transaction_time_min": "neutral",  # neutral, we can ignore it
    "travel_time_min": "lower",  # lower is better
    "total_distance_km": "lower",  # lower is better
    "total_order": "higher",  # higher is better
    "total_qty": "higher",  # higher is better
    "total_volume": "lower",  # lower is better
    "total_weight": "neutral",  # neutral
}


# Extend evaluation_data with percentage and "better or worse" interpretation
evaluation_data = {
    "Metric": [],
    "Quantum": [],
    "Locus": [],
    "Difference": [],
    "Quantum vs Locus (%)": [],
    "Performance": []
}

for metric in direction_rules.keys():
    q = quantum_total.get(metric, None)
    l = locus_total.get(metric, None)

    if q is None or l is None or l == 0:
        continue

    q_rounded = round(q, 2)
    l_rounded = round(l, 2)
    diff = round(q_rounded - l_rounded, 2)

    # Determine percentage comparison based on whether lower or higher is better
    if direction_rules[metric] == "lower":
        # Lower is better: Calculate how much better Quantum is than Locus
        percent_better = round(((l_rounded - q_rounded) / l_rounded) * 100, 2)
        if percent_better > 0:
            performance = f"{percent_better}% better"
        elif percent_better < 0:
            performance = f"{abs(percent_better)}% worse"
        else:
            performance = "Same"
    elif direction_rules[metric] == "higher":
        # Higher is better: Calculate how much better Quantum is than Locus
        percent_better = round(((q_rounded - l_rounded) / l_rounded) * 100, 2)
        if percent_better > 0:
            performance = f"{percent_better}% better"
        elif percent_better < 0:
            performance = f"{abs(percent_better)}% worse"
        else:
            performance = "Same"
    else:
        performance = "Neutral"
        percent_better = 0

    evaluation_data["Metric"].append(metric)
    evaluation_data["Quantum"].append(q_rounded)
    evaluation_data["Locus"].append(l_rounded)
    evaluation_data["Difference"].append(diff)
    evaluation_data["Quantum vs Locus (%)"].append(f"{percent_better}%")
    evaluation_data["Performance"].append(performance)

# Create DataFrame
evaluation_df = pd.DataFrame(evaluation_data)

# Save to Excel
evaluation_df.to_excel("/content/quantum_vs_locus_evaluation_with_percentage_comparison_corrected.xlsx", index=False)

# Optional: Display
print(evaluation_df)


                   Metric  Quantum    Locus  Difference Quantum vs Locus (%)  \
0  total_working_time_min  1636.00  1877.00     -241.00               12.84%   
1    transaction_time_min   734.00   725.00        9.00                   0%   
2         travel_time_min   902.00  1152.00     -250.00                21.7%   
3       total_distance_km   560.09   575.72      -15.63                2.71%   
4             total_order   221.00   219.00        2.00                0.91%   
5               total_qty  4798.00  4773.00       25.00                0.52%   
6            total_volume    14.72    20.89       -6.17               29.54%   
7            total_weight  5476.25  5468.35        7.90                   0%   

     Performance  
0  12.84% better  
1        Neutral  
2   21.7% better  
3   2.71% better  
4   0.91% better  
5   0.52% better  
6  29.54% better  
7        Neutral  


In [ ]:
# Direction rules dictionary with descriptions
direction_rules = {
    "total_working_time_min": {"type": "lower", "description": "Lower is better - Less time spent working is more efficient."},
    "transaction_time_min": {"type": "neutral", "description": "Neutral - Transaction time is the process time, not directly a metric to optimize."},
    "travel_time_min": {"type": "lower", "description": "Lower is better - Less travel time is more efficient."},
    "total_distance_km": {"type": "lower", "description": "Lower is better - Shorter distance is more efficient."},
    "total_order": {"type": "higher", "description": "Higher is better - More orders indicates better performance."},
    "total_qty": {"type": "higher", "description": "Higher is better - More quantity indicates better throughput."},
    "total_volume": {"type": "lower", "description": "Lower is better - Less volume indicates better packing efficiency."},
    "total_weight": {"type": "neutral", "description": "Neutral - Weight is a given factor, and higher is not necessarily better."},
}

# Extend evaluation_data with percentage and "better or worse" interpretation
evaluation_data = {
    "Metric": [],
    "Quantum": [],
    "Locus": [],
    "Difference": [],
    "Quantum vs Locus (%)": [],
    "Performance": [],
    "Description": []
}

for metric, rule in direction_rules.items():
    q = quantum_total.get(metric, None)
    l = locus_total.get(metric, None)

    if q is None or l is None or l == 0:
        continue

    q_rounded = round(q, 2)
    l_rounded = round(l, 2)
    diff = round(q_rounded - l_rounded, 2)

    # Determine percentage comparison based on whether lower or higher is better
    if rule["type"] == "lower":
        # Lower is better: Calculate how much better Quantum is than Locus
        percent_better = round(((l_rounded - q_rounded) / l_rounded) * 100, 2)
        if percent_better > 0:
            performance = f"{percent_better}% better"
        elif percent_better < 0:
            performance = f"{abs(percent_better)}% worse"
        else:
            performance = "Same"
    elif rule["type"] == "higher":
        # Higher is better: Calculate how much better Quantum is than Locus
        percent_better = round(((q_rounded - l_rounded) / l_rounded) * 100, 2)
        if percent_better > 0:
            performance = f"{percent_better}% better"
        elif percent_better < 0:
            performance = f"{abs(percent_better)}% worse"
        else:
            performance = "Same"
    else:
        performance = "Neutral"
        percent_better = 0

    # Add to evaluation data
    evaluation_data["Metric"].append(metric)
    evaluation_data["Quantum"].append(q_rounded)
    evaluation_data["Locus"].append(l_rounded)
    evaluation_data["Difference"].append(diff)
    evaluation_data["Quantum vs Locus (%)"].append(f"{percent_better}%")
    evaluation_data["Performance"].append(performance)
    evaluation_data["Description"].append(rule["description"])

# Create DataFrame
evaluation_df = pd.DataFrame(evaluation_data)

# Save to Excel
evaluation_df.to_excel("/content/quantum_vs_locus_evaluation_with_descriptions.xlsx", index=False)

# Optional: Display
print(evaluation_df)


                   Metric  Quantum    Locus  Difference Quantum vs Locus (%)  \
0  total_working_time_min  1636.00  1877.00     -241.00               12.84%   
1    transaction_time_min   734.00   725.00        9.00                   0%   
2         travel_time_min   902.00  1152.00     -250.00                21.7%   
3       total_distance_km   560.09   575.72      -15.63                2.71%   
4             total_order   221.00   219.00        2.00                0.91%   
5               total_qty  4798.00  4773.00       25.00                0.52%   
6            total_volume    14.72    20.89       -6.17               29.54%   
7            total_weight  5476.25  5468.35        7.90                   0%   

     Performance                                        Description  
0  12.84% better  Lower is better - Less time spent working is m...  
1        Neutral  Neutral - Transaction time is the process time...  
2   21.7% better  Lower is better - Less travel time is more eff...  

In [ ]:
# Direction rules dictionary with updated descriptions
direction_rules = {
    "total_order": {"type": "higher", "description": "Higher is better - More orders indicates better performance."},
    "total_qty": {"type": "higher", "description": "Higher is better - More quantity indicates better throughput."},
    "total_weight": {"type": "higher", "description": "Higher is better - More weight indicates better performance, as it reflects more items delivered."},
    "total_volume": {"type": "lower", "description": "Lower is better - Less volume indicates better packing efficiency."},
    "total_working_time_min": {"type": "lower", "description": "Lower is better - Less time spent working is more efficient."},
    "transaction_time_min": {"type": "lower", "description": "Lower is better - Shorter transaction time is better, but this is a given metric, so slight differences may occur."},
    "travel_time_min": {"type": "lower", "description": "Lower is better - Less travel time is more efficient."},
    "total_distance_km": {"type": "lower", "description": "Lower is better - Shorter distance is more efficient."},

}

# Extend evaluation_data with percentage and "better or worse" interpretation
evaluation_data = {
    "Metric": [],
    "Quantum": [],
    "Locus": [],
    "Difference": [],
    "Quantum vs Locus (%)": [],
    "Performance": [],
    "Description": []
}

for metric, rule in direction_rules.items():
    q = quantum_total.get(metric, None)
    l = locus_total.get(metric, None)

    if q is None or l is None or l == 0:
        continue

    q_rounded = round(q, 2)
    l_rounded = round(l, 2)
    diff = round(q_rounded - l_rounded, 2)

    # Determine percentage comparison based on whether lower or higher is better
    if rule["type"] == "lower":
        # Lower is better: Calculate how much better Quantum is than Locus
        percent_better = round(((l_rounded - q_rounded) / l_rounded) * 100, 2)
        if percent_better > 0:
            performance = f"{percent_better}% better"
        elif percent_better < 0:
            performance = f"{abs(percent_better)}% worse"
        else:
            performance = "Same"
    elif rule["type"] == "higher":
        # Higher is better: Calculate how much better Quantum is than Locus
        percent_better = round(((q_rounded - l_rounded) / l_rounded) * 100, 2)
        if percent_better > 0:
            performance = f"{percent_better}% better"
        elif percent_better < 0:
            performance = f"{abs(percent_better)}% worse"
        else:
            performance = "Same"
    else:
        performance = "Neutral"
        percent_better = 0

    # Add to evaluation data
    evaluation_data["Metric"].append(metric)
    evaluation_data["Quantum"].append(q_rounded)
    evaluation_data["Locus"].append(l_rounded)
    evaluation_data["Difference"].append(diff)
    evaluation_data["Quantum vs Locus (%)"].append(f"{percent_better}%")
    evaluation_data["Performance"].append(performance)
    evaluation_data["Description"].append(rule["description"])

# Create DataFrame
evaluation_df = pd.DataFrame(evaluation_data)

# Save to Excel
evaluation_df.to_excel("/content/quantum_vs_locus_evaluation_with_revised_descriptions.xlsx", index=False)

# Optional: Display
print(evaluation_df)


                   Metric  Quantum    Locus  Difference Quantum vs Locus (%)  \
0             total_order   221.00   219.00        2.00                0.91%   
1               total_qty  4798.00  4773.00       25.00                0.52%   
2            total_weight  5476.25  5468.35        7.90                0.14%   
3            total_volume    14.72    20.89       -6.17               29.54%   
4  total_working_time_min  1636.00  1877.00     -241.00               12.84%   
5    transaction_time_min   734.00   725.00        9.00               -1.24%   
6         travel_time_min   902.00  1152.00     -250.00                21.7%   
7       total_distance_km   560.09   575.72      -15.63                2.71%   

     Performance                                        Description  
0   0.91% better  Higher is better - More orders indicates bette...  
1   0.52% better  Higher is better - More quantity indicates bet...  
2   0.14% better  Higher is better - More weight indicates bette...  